In [1]:
import tkinter as tk
from tkinter import messagebox
import cv2
import numpy as np
from PIL import Image
import os

In [3]:
import mysql.connector

def test_mysql_connection():
    try:
        # Make sure these credentials match your MySQL setup
        mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Gauri@123",  # Replace with your actual MySQL root password
            database="Authorized_user"
        )
        
        print("Connection successful!")
        
        mycursor = mydb.cursor()
        mycursor.execute("SHOW TABLES")
        
        print("\nTables in the database:")
        print("-----------------------")
        for table in mycursor:
            print(table[0])
            
    except mysql.connector.Error as err:
        if err.errno == 1045:
            print("Error: Invalid username or password")
            print("Please verify your credentials")
        elif err.errno == 2003:
            print("Error: MySQL server is not running")
            print("Please start your MySQL server")
        else:
            print(f"Error: {err}")
    finally:
        if 'mydb' in locals() and mydb.is_connected():
            mycursor.close()
            mydb.close()
            print("\nMySQL connection closed.")

test_mysql_connection()

Connection successful!

Tables in the database:
-----------------------
my_table

MySQL connection closed.


In [ ]:
import os
import cv2
import numpy as np
import tkinter as tk
from tkinter import messagebox
from PIL import Image
import mysql.connector

# Create the main window
window = tk.Tk()
window.title("Face Recognition System")

# Labels and input fields for user details
l1 = tk.Label(window, text="Name", font=("Algerian", 20))
l1.grid(column=0, row=0)
t1 = tk.Entry(window, width=50, bd=5)
t1.grid(column=1, row=0)

l2 = tk.Label(window, text="Age", font=("Algerian", 20))
l2.grid(column=0, row=1)
t2 = tk.Entry(window, width=50, bd=5)
t2.grid(column=1, row=1)

l3 = tk.Label(window, text="Address", font=("Algerian", 20))
l3.grid(column=0, row=2)
t3 = tk.Entry(window, width=50, bd=5)
t3.grid(column=1, row=2)

# Train the face recognition model
def train_classifier():
    data_dir = "/Users/gauribarot/Desktop/face recognizer"
    if not os.path.exists(data_dir):
        messagebox.showerror("Error", "Dataset folder not found!")
        return

    faces, ids = [], []
    for file in os.listdir(data_dir):
        image_path = os.path.join(data_dir, file)
        img = Image.open(image_path).convert('L')
        imageNp = np.array(img, 'uint8')
        try:
            id = int(file.split(".")[1])  # Ensure valid ID format
        except ValueError:
            continue
        faces.append(imageNp)
        ids.append(id)

    ids = np.array(ids)

    if len(faces) == 0:
        messagebox.showerror("Error", "No images found for training!")
        return

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")
    messagebox.showinfo('Result', 'Training dataset completed!')

b1 = tk.Button(window, text="Training", font=("Algerian", 20), bg='orange', fg='red', command=train_classifier)
b1.grid(column=0, row=4)

# Face detection and recognition function
def detect_face():
    if not os.path.exists("classifier.xml"):
        messagebox.showerror("Error", "Classifier model not found!")
        return

    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    video_capture = cv2.VideoCapture(0)

    while True:
        ret, img = video_capture.read()
        if not ret:
            break

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 255), 2)
            face_roi = gray[y:y+h, x:x+w]
            id, confidence = clf.predict(face_roi)

            # Correct confidence calculation
            confidence = 100 - confidence  # Lower confidence means a better match

            # Connect to database to get name based on ID
            try:
                mydb = mysql.connector.connect(
                    host="localhost",
                    user="root",
                    password="Gauri@123",
                    database="Authorized_user"
                )
                mycursor = mydb.cursor()
                mycursor.execute(f"SELECT Name FROM my_table WHERE id={id}")
                s = mycursor.fetchone()
                name = s[0] if s else "Unknown"
            except Exception as e:
                name = "Unknown"

            # Show the name and confidence score
            cv2.putText(img, f"{name} ({confidence:.2f}%)", (x, y - 5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 1, cv2.LINE_AA)

        cv2.imshow("Face Recognition", img)
        if cv2.waitKey(1) == 13:  # Enter key
            break

    video_capture.release()
    cv2.destroyAllWindows()

b2 = tk.Button(window, text="Detect the face", font=("Algerian", 20), bg='green', fg='white', command=detect_face)
b2.grid(column=1, row=4)

# Generate dataset for training
def generate_dataset():
    if t1.get() == "" or t2.get() == "" or t3.get() == "":
        messagebox.showinfo('Result', 'Please provide complete details of the user')
    else:
        mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Gauri@123",
            database="Authorized_user"
        )
        mycursor = mydb.cursor()
        mycursor.execute("SELECT * from my_table")
        myresult = mycursor.fetchall()
        id = 1
        for x in myresult:
            id += 1
        sql = "INSERT INTO my_table(id, Name, Age, Address) VALUES(%s, %s, %s, %s)"
        val = (id, t1.get(), t2.get(), t3.get())
        mycursor.execute(sql, val)
        mydb.commit()

        # Capturing face images for the user
        face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        
        def face_cropped(img):
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray, 1.3, 5)
            if len(faces) == 0:
                return None
            for (x, y, w, h) in faces:
                cropped_face = img[y:y+h, x:x+w]
            return cropped_face

        cap = cv2.VideoCapture(0)
        img_id = 0

        while True:
            ret, frame = cap.read()
            if face_cropped(frame) is not None:
                img_id += 1
                face = cv2.resize(face_cropped(frame), (200, 200))
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                file_name_path = f"data/user.{id}.{img_id}.jpg"
                cv2.imwrite(file_name_path, face)
                cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                cv2.imshow("Cropped face", face)
                if cv2.waitKey(1) == 13 or img_id == 200:
                    break
        cap.release()
        cv2.destroyAllWindows()
        messagebox.showinfo('Result', 'Generating dataset completed!')

b3 = tk.Button(window, text="Generate dataset", font=("Algerian", 20), bg='pink', fg='black', command=generate_dataset)
b3.grid(column=2, row=4)

# Set window size and show
window.geometry("800x200")
window.mainloop()


2025-03-27 12:05:51.975 python[15662:697334] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-27 12:05:51.975 python[15662:697334] +[IMKInputSession subclass]: chose IMKInputSession_Modern
